# Calculator

This notebook shows how to create a simple calculator chain with `Kork`.

In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.insert(0, "../")

In [2]:
import math
import operator

from langchain.llms import OpenAI

from kork import CodeChain
from kork.parser import parse

In [4]:
examples = [
    ("calculate the sqrt of 2", "let result = pow(2, 0.5)"),
    ("2*5 + 1", "let result = 2 * 5 + 1"),
    ("1.3e-3", "let result = 1.3 * pow(10, -3)"),
    ("2**5", "let result = pow(2, 5)"),
    ("calculate log of 2", "let result = log2(2)"),
    (
        "every day i eat 3 donuts. how many donuts do i eat during the week",
        "let days_in_week = 7; let result = days_in_week * 3;",
    ),
    ("is 2 > 1?", "let result = gt(2, 1);"),
]

In [5]:
examples_in_ast = [(query, parse(code)) for query, code in examples]

In [11]:
llm = OpenAI(
    model_name="text-davinci-003",
    temperature=0,
    max_tokens=2000,
    frequency_penalty=0,
    presence_penalty=0,
    top_p=1.0,
    verbose=True,
)

chain = CodeChain.from_defaults(
    llm=llm,
    examples=examples_in_ast,
    context=[
        math.sin,
        math.sinh,
        math.asin,
        math.atan,
        math.acos,
        math.cos,
        math.cosh,
        math.tan,
        math.tanh,
        math.ceil,
        math.floor,
        math.dist,
        math.degrees,
        math.radians,
        math.exp,
        math.log10,
        math.log2,
        math.pow,
        operator.ge,
        operator.le,
        operator.eq,
        operator.gt,
        operator.lt,
    ],
)

## Let's test!

In [15]:
queries_and_answers = [
    ("calculate the sin of 37 degrees", math.sin(math.radians(37))),
    ("whats the square root of 10382", math.sqrt(10382)),
    ("what is 2+3**3", 2 + 3**3),
    ("i have 2 halves how much do i have together?", 1),
    ("is 2**3 larger than sin(x) for x = 0.3 radians?", 2**3 > math.sin(0.3)),
    ("i weigh 180 lbs how much do i weight in kgs?", 180 * 0.453592),
]

In [20]:
from kork.display import as_html_dict, display_html_results

In [19]:
html_results = []
for query, answer in queries_and_answers:
    code_result = chain(inputs={"query": query})
    html_results.append(as_html_dict(code_result, query=query, expected_answer=answer))

In [21]:
display_html_results(html_results)

,query,code,result,expected,correct,errors,raw
0,calculate the sin of 37degrees,var result = sin(radians(37)),0.6018150231520483,0.6018150231520483,✅,[],```😼var result = sin(radians(37))```
1,whats the square root of 10382,"var result = pow(10382, 0.5)",101.89209979188769,101.89209979188769,✅,[],"```😼var result = pow(10382, 0.5)```"
2,what is 2+3**3,"var result = 2 + pow(3, 3)",29.0,29,✅,[],"```😼var result = 2 + pow(3, 3)```"
3,i have 2 halves how much do ihave together?,var result = 2 * 1,2,1,⛔,[],```😼var result = 2 * 1```
4,is 2**3 larger than sin(x) forx = 0.3 radians?,"var x = 0.3var result = gt(pow(2, 3), sin(x))",True,True,✅,[],"```😼var x = 0.3var result = gt(pow(2, 3), sin(x))```"
5,i weigh 180 lbs how much do iweight in kgs?,var lbs_to_kgs = 0.453592var result = lbs_to_kgs * 180,81.64656,81.64656,✅,[],```😼var lbs_to_kgs = 0.453592var result = lbs_to_kgs * 180```
